[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WhoIsLTD/naive_bayes/blob/master/naive_b.ipynb)



In [9]:
import pandas as pd
import sklearn
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [10]:
df = pd.read_csv('data.csv')
df


,Quang cảnh,Nhiệt độ,Độ ẩm,Gió,Chơi
0,Nắng,Nóng,Cao,Nhẹ,Không
1,Nắng,Nóng,Cao,Mạnh,Không
2,Âm u,Nóng,Cao,Nhẹ,Có
3,Mưa,Ấm áp,Cao,Nhẹ,Có
4,Mưa,Mát,Trung bình,Nhẹ,Có
5,Mưa,Mát,Trung bình,Mạnh,Không
6,Âm u,Mát,Trung bình,Mạnh,Có
7,Nắng,Ấm áp,Cao,Nhẹ,Không
8,Nắng,Mát,Trung bình,Nhẹ,Có
9,Mưa,Ấm áp,Trung bình,Nhẹ,Có


In [11]:
X = df.drop(['Chơi'], axis=1)
y = df.Chơi

In [12]:
X

,Quang cảnh,Nhiệt độ,Độ ẩm,Gió
0,Nắng,Nóng,Cao,Nhẹ
1,Nắng,Nóng,Cao,Mạnh
2,Âm u,Nóng,Cao,Nhẹ
3,Mưa,Ấm áp,Cao,Nhẹ
4,Mưa,Mát,Trung bình,Nhẹ
5,Mưa,Mát,Trung bình,Mạnh
6,Âm u,Mát,Trung bình,Mạnh
7,Nắng,Ấm áp,Cao,Nhẹ
8,Nắng,Mát,Trung bình,Nhẹ
9,Mưa,Ấm áp,Trung bình,Nhẹ


In [13]:
y

0     Không
1     Không
2        Có
3        Có
4        Có
5     Không
6        Có
7     Không
8        Có
9        Có
10       Có
11       Có
12       Có
13    Không
Name: Chơi, dtype: object

In [45]:
from sklearn.preprocessing import OrdinalEncoder

# Cols need to use ordinal encoder
cols = X.columns

# Apply ordinal encoder to each column in cols
ordinal_encoder = OrdinalEncoder()
X[cols] = ordinal_encoder.fit_transform(X[cols])
y = ordinal_encoder.fit_transform(np.array(y).reshape(-1, 1))
df[list(df.columns)] = ordinal_encoder.fit_transform(df[list(df.columns)])

In [ ]:
nb = GaussianNB()
nb.fit(X, y)
sklearn_preds = nb.predict(X)

# print(f"sklearn accuracy:{accuracy_score(y, sklearn_preds)}")
print(f"predictions: {sklearn_preds}")

predictions: [1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
df

,Quang cảnh,Nhiệt độ,Độ ẩm,Gió,Chơi
0,1.0,1.0,0.0,1.0,1.0
1,1.0,1.0,0.0,0.0,1.0
2,2.0,1.0,0.0,1.0,0.0
3,0.0,2.0,0.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0
5,0.0,0.0,1.0,0.0,1.0
6,2.0,0.0,1.0,0.0,0.0
7,1.0,2.0,0.0,1.0,1.0
8,1.0,0.0,1.0,1.0,0.0
9,0.0,2.0,1.0,1.0,0.0


In [19]:
n_sample, n_features = X.shape
n_classes = len(np.unique(y))
print(n_sample, n_features, n_classes)

14 4 2


In [20]:
def cal_prior(df, y):
    """Calculate P(y) for all possible y"""
    classes = np.unique(y)
    prior = []
    for i in classes:
        prior.append(len(df[y==i])/len(df))
    return prior

In [23]:
def cal_likelihood_gaussian(df, feature_name, feature_val, y, label):
    features = list(df.columns)
    df = df[y==label]
    mean, std = df[feature_name].mean(), df[feature_name].std()
    P_x_given_y = (1 / (np.sqrt(2 * np.pi) * std)) *  np.exp(-((feature_val-mean)**2 / (2 * std**2 )))
    return P_x_given_y

In [54]:
def naive_bayes_gaussian(df, X, y):
    """Calculate P(X=x1|Y=y)P(X=x2|Y=y)...P(X=xn|Y=y) * P(Y=y) 
    for all y and find the maximum"""
    features = list(X.columns)
    prior = cal_prior(df,y)
    y_preds = []
    for x in np.array(X):
        labels = sorted(list(np.unique(y)))
        likelihood = [1]*len(labels)
        for i in range(len(labels)):
            for j in range(len(features)):
                likelihood[i] *= cal_likelihood_gaussian(df, features[j], x[j], y, labels[i])
        post_prob = [1]*len(labels)
        for i in range(len(labels)):
            post_prob[i] = likelihood[i]*prior[i]
        print(post_prob)
        y_preds.append(np.argmax(post_prob))
    return y_preds

In [55]:
Y_pred = naive_bayes_gaussian(df, X, y)

[0.024545939914561037, 0.029790002965039197]
[0.012602305754881034, 0.04157529828136013]
[0.01562595880212199, 0.0014831569142992778]
[0.007684890571990846, 0.013905272772414153]
[0.011563554379488557, 0.0017521428646211314]
[0.005936926775293726, 0.002445312352391184]
[0.007684890571990847, 0.00016990874156188712]
[0.01562595880212199, 0.019406371455392486]
[0.023512582599230898, 0.0024453123523911838]
[0.014968122968846712, 0.003102685740616372]
[0.015625958802121993, 0.006043206635693443]
[0.005107213678886668, 0.0013484216644323332]
[0.03043521188301095, 0.000330937039812849]
[0.0039455543775522685, 0.019406371455392483]


In [51]:
Y_pred

[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]

In [50]:
Y_pred == sklearn_preds

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])